# Introduction to Performance in Python

**Author: Simone Bacchio**

This notebook is part of the Beginner Training event 2022 by NCC Cyprus.

Details on the training can be found at https://castorc.cyi.ac.cy/hpcb2022

Presentations and source code is available at https://github.com/CaSToRC-CyI/NCC-Beginner-Training-2022


## Performance

It is very important to understand performanc limitations in Python and how to overcome them.

In this section we analyze the performance of a very simple operation: the addition of two arrays

$$z = x + y$$

In [ ]:
def add1(x1, x2):
    "Add with direct memory access and list extension"
    y = []
    for i in range(len(x1)):
        y.append(x1[i] + x2[i])
    return y


def add2(x1, x2):
    "Add with indirect memory access and list extension"
    y = []
    for i1, i2 in zip(x1, x2):
        y.append(i1 + i2)
    return y


def add3(x1, x2):
    "Add with indirect memory access and list generation"
    return [i1 + i2 for i1, i2 in zip(x1, x2)]


def add4(x1, x2):
    "Add with buit-in numpy function"
    return x1 + x2

In [ ]:
import numpy as np
from timeit import timeit
from pandas import DataFrame

# Running over sizes that are powers of 2
times = DataFrame(index=[2 ** i for i in range(12)])

# Collecting data for the 4 types of add
for size in times.index:
    x1, x2 = np.random.rand(2, size)
    for add in [
        add1,
        add2,
        add3,
        add4,
    ]:
        times.at[size, add.__name__] = timeit(lambda: add(x1, x2), number=1000)

In [ ]:
# Dropping column 4 for not ruining the surprise
times.drop(columns=["add4"])

In [ ]:
times.drop(columns=["add4"]).plot(ylabel="Time x1000 [seconds]", xlabel="Size")

In [ ]:
times.plot(ylabel="Time x1000 [seconds]", xlabel="Size")

In [ ]:
times.plot(logx=True, logy=True, ylabel="Time x1000 [seconds]", xlabel="Size")

In [ ]:
perf = times.apply(lambda x: np.array(times.index) / x)
perf.plot(logx=True, logy=True, ylabel="kFLOPS", xlabel="Size")

In [ ]:
times = DataFrame(index=[2 ** i for i in range(20)])

for size in times.index:
    x1, x2 = np.random.rand(2, size)
    for add in [
        add4,
    ]:
        times.at[size, add.__name__] = timeit(lambda: add(x1, x2), number=1000)

times.plot(logx=True, logy=True, ylabel="Time x1000 [seconds]", xlabel="Size")

In [ ]:
perf = times.apply(lambda x: np.array(times.index) / x)
perf.plot(logx=True, logy=True, ylabel="kFLOPS", xlabel="Size")

## Introduction to Numpy

In [ ]:
import numpy as np

### Array creation

An array can be easily created from a list of data

In [ ]:
arr = np.array([1,2,3,4])
arr

`shape` and `dtype` are deduced from the data itself

In [ ]:
arr.shape

In [ ]:
arr.dtype

These paramters can be changed respectively with `reshape` and `astype`

In [ ]:
arr.reshape(2,2)

In [ ]:
arr.astype("int32")

### How does it work?

Nd-arrays are always a "view" of **unrolled data**.

- Data are stored sequentially in a C-style pointer and processed all at once
- The Nd view of the pointer allows from high dynamism on the the Python side

In [ ]:
arr.__array_interface__

In [ ]:
arr2 = arr.reshape(2,2)
arr2

In [ ]:
arr2.__array_interface__

Reshaped data indeed share the same pointer! But they are two different objects in Python

In [ ]:
arr is arr2

In [ ]:
arr == arr2

Numpy tries as most as possible to not create copies of arrays (e.g. with `transpose` in the following) and you need to be well aware of this because editing arrays in-place might change the content also of other arrays!

In [ ]:
arr3 = arr2.transpose()
arr3

In [ ]:
arr3.__array_interface__

## Item accessing

In [ ]:
flat = np.arange(90)

In [ ]:
arr = flat.reshape(10,9)
arr

The indeces go from slowest (left) to fastest (right) running indeces.

E.g. for a matrix the first selects a row

In [ ]:
arr[0]

and the second index selects a column

In [ ]:
arr[:,0]

The colon symbol `:`, alone or with number left OR right, is a slice, namely a range with open end(s).

A colon with numbers left AND right is a range.

In [ ]:
arr[2:4, :4]

`None` can be used for inserting new axes and `...` (`Ellipsis`) to dynamically skip axes. E.g.

In [ ]:
arr[None].shape

In [ ]:
arr[..., None].shape

Setting values follows the same rules, but needs to be followed by assignment, e.g.

In [ ]:
arr[2:5, 4:6]=0
arr

**Note** that this changes values IN-PLACE, and therefore any array pointing to the same data are changed. E.g. the original flat array has also change

In [ ]:
flat

Same is true for operations that assign value, e.g. `+=`, `-=`, `*=`, `/=`, ...

In [ ]:
arr+=1
arr

In [ ]:
flat

### Exercise

Write two functions, `get_rows` and `get_columns`, that get a two dimensional array as parameter. They should return the list of rows and columns of the array, respectively. The rows and columns should be one dimensional arrays. You may use the transpose operation, which flips rows to columns, in your solution. The transpose is done by the `T` method:

In [ ]:
print(arr)
print(arr.T)

In [ ]:
def get_row(arr):
    "Returns list of rows of a matrix"
    # Complete
    return arr

def get_columns(arr):
    "Returns list of columns of a matrix"
    # Complete
    return arr
    

**Extra:** Create function `get_row_vectors` that returns a list of rows from the input array of shape `(n,m)`, but this time the rows must have shape `(1,m)`. Similarly, create function `get_columns_vectors` that returns a list of columns (each having shape `(n,1)`) of the input matrix .

**Example:** for a 2x3 input matrix
```python
[[5 0 3]
 [3 7 9]]
```
the result should be
```python
Row vectors:
[array([[5, 0, 3]]), array([[3, 7, 9]])]
Column vectors:
[array([[5],
        [3]]),
 array([[0],
        [7]]),
 array([[3],
        [9]])]
```

In [ ]:
def get_row_vector(arr):
    "Returns list of row-vectors of a matrix"
    # Complete
    return arr

def get_columns_vector(arr):
    "Returns list of column-vectors of a matrix"
    # Complete
    return arr

## Vectorized Universal functions

Element-wise functions are called universal functions.

These can be 

- unary functions, e.g. `np.sqrt`, `np.exp`, `np.cos`, ...
- binary functions, e.g. `+` or `np.add`, `*` or `np.mul`, ...
- reduction functions, e.g. `np.sum`, `np.prod`, `np.min`, ...



In [ ]:
arr1 = np.ones(10)

# E.g.
arr2 = arr1 + 2
arr3 = arr1 + arr2
np.sqrt(arr3)

In [ ]:
arr3.sum()

Very important is the keyword `axis` that allows to specify the axis (or axes) along which perform an operation.

```python
# creating a shape-(3,2) array
>>> x = np.array([[0, 1],
...               [2, 3],
...               [4, 5]])

# sum over all axes
>>> np.sum(x)  # equivalent: x.sum(axis=0)
15

# sum over axis-0, within axis-1
# i.e. sum over the rows, within each column
>>> np.sum(x, axis=0)  # equivalent: x.sum(axis=0)
array([6, 9])

# sum over axis-1, within axis-0
# i.e. sum over the columns, within each row
>>> np.sum(x, axis=1)  # equivalent: x.sum(axis=1)
array([1, 5, 9])

# negative axis-indices can be used too
>>> np.sum(x, axis=-1)  # equivalent: np.sum(x, axis=1)
array([1, 5, 9])

# sum over axis-0 and axis-1
# i.e. sum the array as if it were a 1D sequence (default behavior)
>>> np.sum(x, axis=(0, 1))  # equivalent: x.sum(axis=(0, 1))
15
```

### Exercise:

Let $x$ and $y$ be m-dimensional vectors. The angle $\alpha$ between two vectors is defined by the equation 
$$\cos_{xy}(\alpha) = \frac{<x,y>}{|x||y|},$$
where the angle brackets denote the inner product, and $|x| = \sqrt{<x,x>}$.

Write function `vector_angles` that gets two arrays `X` and `Y` with same shape `(n,m)` as parameters. Each row in the arrays corresponds to a vector. The function should return vector of shape `(n,)` with the corresponding angles between vectors of `X` and `Y` in degrees, not in radians. Don’t use loops, but use vectorized operations.

In [ ]:
def cosxy(x,y):
    "Returns the cos of the angle between two vectors"
    # Compute the norms xx, yy, xy
    return xy/(xx*yy)**0.5

## Broadcasting

In [ ]:
arr1 = np.arange(3)
arr1

In [ ]:
arr2 = np.zeros((3,3))
arr2

In [ ]:
arr1+arr2

In [ ]:
arr3 = arr1[:,None]
arr3

In [ ]:
arr2 + arr3

In [ ]:
arr1.shape, arr3.shape, (arr1+arr3).shape

In [ ]:
arr1+arr3

In [ ]:
x = np.arange(1,11)
x

In [ ]:
prods = x * x[:, None]
prods

## Additional information and Numba

E.g. how to perfmorm `x[i+1] - x[i]` ?

In [ ]:
arr = np.arange(10)

N = len(arr)
out = np.zeros_like(arr)
for i in range(N):
    out[i] = arr[(i+1)%N] - arr[i]
out

In [ ]:
arr

In [ ]:
np.roll(arr, -1)

In [ ]:
def difference(arr):
    return np.roll(arr, -1) - arr

In [ ]:
from numba import njit, prange

@njit#(parallel=True)
def difference_numba(arr):
    N = arr.shape[0]
    out = np.empty_like(arr)
    for i in range(N):
        out[i] = arr[(i+1)%N] - arr[i]
    return out

@njit(parallel=True)
def difference_numba_par(arr):
    N = arr.shape[0]
    out = np.empty_like(arr)
    for i in prange(N):
        out[i] = arr[(i+1)%N] - arr[i]
    return out



In [ ]:
difference(np.arange(10))

In [ ]:
difference_numba(np.arange(10))

In [ ]:
times = DataFrame(index=[2 ** i for i in range(20)])

for size in times.index:
    x = np.random.rand(1, size)
    for fnc in [
        difference,
        difference_numba,
        difference_numba_par,
    ]:
        times.at[size, fnc.__name__] = timeit(lambda: fnc(x), number=1000)

times.plot(logx=True, logy=True, ylabel="Time x1000 [seconds]", xlabel="Size")

In [ ]:
times.plot(ylabel="Time x1000 [seconds]", xlabel="Size")